<a href="https://colab.research.google.com/github/SaatvikP/News_Classification/blob/main/News_Classification_20_Groups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install tensorflow numpy scikit-learn


In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import gensim.downloader as api


In [22]:
# Load Dataset (20 Newsgroups)
categories = [
    'alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware',
    'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
    'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med',
    'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast',
    'talk.politics.misc', 'talk.religion.misc'
]

newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))
X_texts = newsgroups.data  # List of news articles
y_labels = newsgroups.target  # Numerical labels
category_labels = newsgroups.target_names  # Label names

# Split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X_texts, y_labels, test_size=0.2, random_state=42)


In [23]:
MAX_VOCAB_SIZE = 20000  # Max number of unique words
MAX_LENGTH = 200  # Max words per document

# Tokenization
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Convert labels to categorical (One-hot encoding)
num_classes = len(category_labels)
y_train_enc = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_enc = tf.keras.utils.to_categorical(y_test, num_classes)

In [24]:
print(" Loading GloVe embeddings...")
glove_vectors = api.load("glove-wiki-gigaword-100")

def create_embedding_matrix(tokenizer, embedding_dim=100):
    vocab_size = min(MAX_VOCAB_SIZE, len(tokenizer.word_index) + 1)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for word, i in tokenizer.word_index.items():
        if i >= MAX_VOCAB_SIZE:
            continue
        if word in glove_vectors:
            embedding_matrix[i] = glove_vectors[word]

    return embedding_matrix

embedding_matrix = create_embedding_matrix(tokenizer, embedding_dim=100)


 Loading GloVe embeddings...


In [25]:
# BiLSTM Model (BiLSTM)
def build_model():
    lstm_units = 128
    dense_units = 64

    inputs = Input(shape=(MAX_LENGTH,))
    x = Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=100,
                  weights=[embedding_matrix], trainable=False)(inputs)

    x = Bidirectional(LSTM(lstm_units, dropout=0.4, return_sequences=False))(x)

    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(0.5)(x)

    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

print(" Building Model...")
model = build_model()
model.summary()


 Building Model...


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 200, 100)            │       2,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 256)                 │         234,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 20)                  │           1,300 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,252,244 (8.59 MB)

 Trainable params: 252,244 (985.33 KB)

 Non-trainable params: 2,000,000 (7.63 MB)

In [26]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

print(" Training Model...")
history = model.fit(
    X_train_padded, y_train_enc,
    validation_split=0.1,
    epochs=10, batch_size=32,
    callbacks=[early_stop]
)

 Training Model...
Epoch 1/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.0789 - loss: 2.9512 - val_accuracy: 0.1824 - val_loss: 2.4413
Epoch 2/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.2063 - loss: 2.3933 - val_accuracy: 0.3276 - val_loss: 1.9437
Epoch 3/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.3083 - loss: 2.0315 - val_accuracy: 0.4158 - val_loss: 1.7283
Epoch 4/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.3879 - loss: 1.8095 - val_accuracy: 0.4569 - val_loss: 1.6091
Epoch 5/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.4471 - loss: 1.6831 - val_accuracy: 0.4894 - val_loss: 1.5275
Epoch 6/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.4777 - loss: 1.5823 - val_accuracy: 0.5245 - val_loss: 1.4618
Epoch 7/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.5102 - loss: 1.5165 - val_accuracy: 0.5365 - val_loss: 1.4284
Epoch 8/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.5369 - los

In [27]:
# Evaluate Model

print(" Evaluating Model...")
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_enc, axis=1)

print("\n Classification Report:")
print(classification_report(y_true, y_pred, target_names=category_labels))

print("\n Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

 Evaluating Model...
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step

 Classification Report:
                          precision    recall  f1-score   support

             alt.atheism       0.25      0.28      0.26       151
           comp.graphics       0.46      0.54      0.50       202
 comp.os.ms-windows.misc       0.57      0.50      0.53       195
comp.sys.ibm.pc.hardware       0.41      0.39      0.40       183
   comp.sys.mac.hardware       0.53      0.37      0.44       205
          comp.windows.x       0.58      0.56      0.57       215
            misc.forsale       0.63      0.64      0.63       193
               rec.autos       0.42      0.60      0.50       196
         rec.motorcycles       0.54      0.54      0.54       168
      rec.sport.baseball       0.74      0.80      0.77       211
        rec.sport.hockey       0.90      0.80      0.85       198
               sci.crypt       0.70      0.66      0.68       201
         sci.electronics       0.43      0.41      0.4

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [28]:
# Making Predictions
# ============================
def predict_category(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
    pred_probs = model.predict(padded)[0]
    pred_category_idx = np.argmax(pred_probs)
    return category_labels[pred_category_idx]

# Test with some sample news articles
sample_news = [
    "Apple unveils the new MacBook Pro with M3 chip.",
    "The Lakers won a thrilling game in overtime last night.",
    "NASA launches a new Mars rover to explore the red planet.",
    "The US government announces new policies on climate change.",
    "Bitcoin prices are soaring after a new regulation change."
]

print("\n🔍 Testing Predictions...\n")
for text in sample_news:
    print(f"News: {text}\nPredicted Category: {predict_category(text)}\n")


🔍 Testing Predictions...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
News: Apple unveils the new MacBook Pro with M3 chip.
Predicted Category: sci.electronics

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
News: The Lakers won a thrilling game in overtime last night.
Predicted Category: rec.sport.hockey

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
News: NASA launches a new Mars rover to explore the red planet.
Predicted Category: sci.space

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
News: The US government announces new policies on climate change.
Predicted Category: sci.crypt

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
News: Bitcoin prices are soaring after a new regulation change.
Predicted Category: rec.autos

